In [1]:
# !pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
# !pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 761 kB 5.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=64cb9a2bef4570a1fe7d26676ce69a4e4a876d1d52b8e22c22806fc08b5736d8
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [4]:
import flask_ngrok

In [14]:
port_no = 5000

In [13]:
from flask import * 
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

In [16]:
from flask import Flask,request, url_for, redirect, render_template
import pickle
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from bs4 import BeautifulSoup
import requests
from lxml import etree
import re

app = Flask(__name__)
ngrok.set_auth_token("2JDNjznpCvz6dO2azRMYoBOtPSl_3zHP68shUQqtQNjBNNQvJ")
public_url =  ngrok.connect(port_no).public_url
url = 'https://finance.yahoo.com/quote/%5ENSEI/history/'
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \
            (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',\
            'Accept-Language': 'en-US, en;q=0.5'})

page = requests.get(url,headers=HEADERS, data={'ajax': True, 'mobile': False})
soup = BeautifulSoup(page.content,"html.parser")
dom = etree.HTML(str(soup))

model = pickle.load(open('model.pkl','rb'))

def download_nifty_data():
  close_list = []
  for i in range(1,83):
    i = str(i)
    n = dom.xpath('/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr['+i+']/td[6]/span')[0].text
    n = float(re.sub(",","",n))      # Removing commas from closing price and converting it as float datatype
    close_list.append(n)
  return close_list


def create_time_data(dataset):
    X = []
    Y = []
    for i in range(len(dataset)-101):
        a = dataset[i:(i+100),0]
        X.append(a)
        Y.append(dataset[i + 100,0])
    return np.array(X), np.array(Y)

@app.route('/')
def open_site():
    return render_template("index.html")

@app.route('/predict',methods=['POST'])
def make_preds():
    last_data = []
    last_data = download_nifty_data()
    test_df = pd.DataFrame()
    test_df['Close'] = last_data
    scaler = MinMaxScaler(feature_range=(0, 1))
    test_df = test_df.iloc[::-1]  # inverting dataframe
    test_df = scaler.fit_transform(np.array(test_df).reshape(-1, 1))
    X, Y = create_time_data(test_df)  # Converting into X numpy array with 100 features and a Y numpy array.
    print(X.shape)
    X = X.reshape(X.shape[0], X.shape[1], 1)  # Reshaping array for LSTM model
    X = X[-1:]
    fin_pred = model.predict(X)
    fin_pred = scaler.inverse_transform(fin_pred)
    fin_pred = int(fin_pred[0])
    return render_template('index.html', prediction_text='Nifty prediction for tomorrow is: {}'.format(fin_pred))



In [21]:
app.run(port=port_no)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
